In [1]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # ajuste se o caminho for diferente

In [2]:
import sys
print(sys.executable)

C:\Users\niton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [8]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')
print(tokenizer.tokenize("Olá! Isto é um teste. Vamos ver se funciona."))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\niton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\niton\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


['Olá!', 'Isto é um teste.', 'Vamos ver se funciona.']


In [9]:
import os
os.environ['TESSDATA_PREFIX'] = r'C:\Program Files\Tesseract-OCR\tessdata'
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

from PIL import Image
img = Image.new('RGB', (300, 50), color = (255, 255, 255))
# Vai tentar rodar OCR em branco, só pra testar o idioma
print(pytesseract.image_to_string(img, lang='por'))

In [10]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import nltk
nltk.download('punkt')
import os
import mimetypes
from pydub import AudioSegment
import whisper
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Configurações específicas para Windows:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
POPPLER_PATH = r'C:\poppler\bin'  # ajuste conforme sua instalação

whisper_model = whisper.load_model("base")

def transcrever_audio_m4a(path):
    wav_path = path.replace('.m4a', '.wav')
    audio = AudioSegment.from_file(path, format="m4a")
    audio.export(wav_path, format="wav")
    result = whisper_model.transcribe(wav_path, language="portuguese")
    os.remove(wav_path)
    return result["text"]

def extrair_texto_pdf(path):
    doc = fitz.open(path)
    texto = ""
    for page in doc:
        texto += page.get_text()
    doc.close()
    return texto

def extrair_texto_pdf_ocr(path):
    pages = convert_from_path(path, poppler_path=POPPLER_PATH)
    texto_total = ""
    for i, page in enumerate(pages):
        texto = pytesseract.image_to_string(page, lang='por')
        texto_total += f"\n--- Página {i+1} ---\n"
        texto_total += texto
    return texto_total

def resumir_texto(texto, num_sentencas=5):
    parser = PlaintextParser.from_string(texto, Tokenizer("portuguese"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentencas)
    return "\n".join(str(sentence) for sentence in summary)

def arquivo_pdf_tem_texto(path):
    doc = fitz.open(path)
    for page in doc:
        if page.get_text().strip():
            doc.close()
            return True
    doc.close()
    return False

def processar_pasta(pasta):
    todos_textos = []
    for arquivo in os.listdir(pasta):
        caminho = os.path.join(pasta, arquivo)
        if not os.path.isfile(caminho):
            continue

        tipo, _ = mimetypes.guess_type(caminho)
        print(f"\nProcessando: {arquivo} | Tipo: {tipo}")

        # Áudio m4a
        if arquivo.lower().endswith('.m4a'):
            texto = transcrever_audio_m4a(caminho)
            todos_textos.append(f"\n[Áudio: {arquivo}]\n{texto}")

        # PDF
        elif arquivo.lower().endswith('.pdf'):
            if arquivo_pdf_tem_texto(caminho):
                texto = extrair_texto_pdf(caminho)
                todos_textos.append(f"\n[PDF Texto: {arquivo}]\n{texto}")
            else:
                texto = extrair_texto_pdf_ocr(caminho)
                todos_textos.append(f"\n[PDF Imagem: {arquivo}]\n{texto}")

    return "\n".join(todos_textos)

if __name__ == "__main__":
    pasta = input("Digite o caminho da pasta: ").strip()
    textos_extraidos = processar_pasta(pasta)
    print("\n--- Texto completo extraído dos arquivos ---\n")
    print(textos_extraidos[:1000])  # Mostra só os 1000 primeiros caracteres

    print("\n--- RESUMO FINAL ---\n")
    resumo = resumir_texto(textos_extraidos, num_sentencas=8)
    print(resumo[:1000])

    with open("texto_extraido.txt", "w", encoding="utf-8") as f:
        f.write(textos_extraidos)
    with open("resumo_final.txt", "w", encoding="utf-8") as f:
        f.write(resumo)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\niton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\niton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\niton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Processando: download.pdf | Tipo: application/pdf

Processando: métodos analíticos em Keynes_Silva ME.pdf | Tipo: application/pdf

Processando: projeto.ipynb | Tipo: None

--- Texto completo extraído dos arquivos ---


[PDF Texto: download.pdf]
Reflexões sobre o método em Keynes
Reflections on the method of Keynes
Fernando Ferrari Filho 
Fábio Henrique Bittes Terra*
resumo: Provavelmente, a questão do método seja uma das mais controvertidas discussões 
relacionadas à obra de Keynes. As controvérsias podem ser resumidas em três principais 
linhas: (i) o atomismo ou o organicismo como unidade de análise econômica de Keynes; 
(ii) a continuidade ou a descontinuidade das bases filosóficas, em termos epistemológicos e 
metodológicos, do pensamento de Keynes; e (iii) a especulação acerca do método científico 
utilizado por Keynes. O objetivo do artigo centra-se na última das linhas da controvérsia, 
qual seja, discutir o método científico na obra de Keynes. Subjacente a tal objetivo, apesar

1 - instalar phyton abaixo da versão 13
2 - baixar tesseract e FFmpeg
3 - verificar onde o python está instalado na máquina:
        import sys
        print(sys.executable)
4 - pip install pydub openai-whisper pymupdf pdf2image pytesseract pillow sumy dentro do python que está rodando
5 - rodar o código (torcer p dar certo)